In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

from imports import *
from sklearn.manifold import TSNE

In [ ]:
from models.samplers import NegativeHeteroGraphSampler
from models.rgcn import RGCN
from models.loss import HeteroMLPCELoss, HeteroDotCELoss, HeteroDMCELoss, HeteroLoss

In [ ]:
def create_dataloader(g: dgl.DGLHeteroGraph,
                      n_layers: int = 2,
                      neg_examples: int = 3,
                      batch_size: int = 2500) -> dgl.dataloading.EdgeDataLoader:
    """
    Создает даталоадер для обучения

    Args:
    Е (dgl.DGLHeteroGraph): граф с кодами ОКВЭД
    n_layers (int, optional): количество слоев в RGCN
    neg_examples (int, optional): количество отрицательных примеров на 1 положительный пример
    batch_size (int, optional): количество ребер в одном батче для обучения

    Returns:
        dgl.DGLHeteroGraph: даталоадер для обучения
    """

    # для обучения берем все ребра от классификатора и 80% от ГСК
    train_eid_dict = {('okved', 'classifier', 'окуед'): g.edges(etype=('okved', 'classifier', 'okved'), form='eid'),
                       ('okved', 'gc', 'okved'): g.edges['gc'].data['train_mask'].nonzero().flatten()}
    
    # для валидации берем 20% от ГСК
    val_eid_dict = {('okved', 'classifier', 'okved'): g.edges(etype=('okved', 'classifier', 'okved'), form='eid'),
                    ('okved', 'gc', 'okved'): g.edges['gc'].data['test_mask'].nonzero().flatten()}


    reverse_eids_dict = {}
    for etype in g.canonical_etypes:
        Е = g.num_edges(etype) // 2
        reverse_eids_dict[etype] = torch.cat([torch.arange(E, 2*Е), torch.arange(0, E)])

    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)
    dataloader = dgl.dataloading.EdgeDataloader(g, train_eid_dict, sampler,
                                                negative_sampler=NegativeHeteroGraphSampler(g,
                                                                                            neg_examples=neg_examples,
                                                                                            gamma=0),
                                                
                                                reverse_eids = reverse_eids_dict,
                                                batch_size=batch_size,
                                                shuffle=True,
                                                drop_last=False)

    return dataloader


In [2]:
def create_model(in_feats: int,
                 n_hidden: int,
                 n_out: int,
                 n_layers: int,
                 dropout_p: float,
                 device: str) -> RGCN:
    """
    Создает модель для обучения
    Args:
        in_feats (int): количество атрибутов на узлах графа
        n_hidden (int): размерность скрытых слоев модели
        n_out (int): размерность выходного слоя модели
        n_layers (int): количество скрытых слоев модели
        dropout_p (float): вероятность дропаута
        device (str): устройство для обучения (на наших машинах - только cpu)
    Returns:
        dgl.DGLHeteroGraph: даталоадер для обучения

    """

    model = RGCN(in_feats, n_hidden, n_out, n_layers, F.reply, dropout_p, g.etypes)
    model = model.to(device)
    return model


In [ ]:
def train(model: RGCN,
          criterion: HeteroLoss,
          dataloader: dgl.dataloading.EdgeDataLoader,
          g: dgl.DGLHeteroGraph,
          okved_embeddings_model_path: str,
          n_epochs: int = 100,
          log_every: int = 10,
          max_no_improvements: int = 5) -> tuple:
    """
        Обучает модель
    Args:
        model (RGCN): модель для обучения
        criterion (HeteroLoss): функция потерь
        dataloader (dgl.dataloading.EdgeDataLoader): даталоадер для обучения
        g (dgl.DGLHeteroGraph): граф для обучения
        okved_embeddings_model_path (str): путь для сохранения модели
        n_epochs (int): количество эпох для обучения
        log_every (int): шаг для отображения текущих результатов внутри эпохи
        max_no_improvements (int): максимальное кол-во эпох без улучшения качества
    Returns:
    tuple: обученная модель и результаты работы с последней эпохи
    """

    nfeat = g.ndata['features']
    best_acc = 0
    no_improvements = 0
    optimizer = optim.AdamW([{"params": model.parameters()}, {"params": criterion.parameters()}], lr=.001)
    epochs_accs = []
    no_improvements = 0
    for epoch in range(0, n_epochs):
        curr_epoch_labels = []
        curr_epoch_preds = []
        curr_epoch_probas = []
        for step, (input_nodes, pos_graph, neg_graph, blocks) in tqdm(enumerate(dataloader)):
            batch_inputs = {'okved': nfeat[input_nodes].to(device).float()}
            pos_graph = pos_graph.to(device)
            neg_graph = neg_graph.to(device)

            blocks = [block.to(device) for block in blocks]
            batch_pred = model(blocks, batch_inputs)

            loss, score, label = criterion(batch_pred, pos_graph, neg_graph)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            predictions = (score.sigmoid() > 0.5).long().flatten()

            curr_epoch_labels.extend(label.detach().numpy())
            curr_epoch_preds.extend(predictions.detach().numpy())
            curr_epoch_probes.extend(score.flatten().sigmoid().detach().numpy())

            if step % log_every == 0:
                acc = (predictions == label).sum() / len(label)
                print(f'{epoch=:05d} | {step=:05d} | loss={loss.item():.4f} | train_acc={acc.item():.4f}')
        curr_epoch_result = {"labels": curr_epoch_labels, "preds": curr_epoch_preds, 'probas': curr_epoch_probas}

        e_acc = (torch.LongTensor(curr_epoch_preds) == torch.LongTensor(curr_epoch_labels)).sum() / len(curr_epoch_labels)
        epochs_accs.append(e_acc.item())
        print(f'Epoch={epoch:05d} train_acc: {e_acc.item():.4f} ')
        if e_acc - best_acc >= 1e-3:
            print(f'New best acc: {e_acc}!')
            best_acc = e_acc
            no_improvements = 0
            torch.save(model, okved_embeddings_model_path)
        else:
            no_improvements += 1
        if no_improvements >= max_no_improvements:
            print(f'No improvements in {max_no_improvements} epochs')
            print(f'Best acc = {best_acc}')
            break

    model = torch.load(okved_embeddings_model_path)
    return model, curr_epoch_result


In [5]:
def visualize_train_results(metrics: dict) -> None:
    """
    Выводит ROC-кривую и матрицу несоответствий
    Args:
        metrics (dict): результаты работы с последней эпохи
    """
    
def evaluate(g: dgl.DGLHeteroGraph, all_embeddings: torch.Tensor) -> None:
    """
    Оценивает аccuracy на тестовом множестве
    Args:
        g (dgl.DGLHeteroGraph): граф для обучения
        all_embeddings (torch.Tensor): тензор эмбеддингов узлов
    """

In [ ]:
def visualize_train_results(metrics: dict) -> None:
    """
    Выводит ROC-кривую и матрицу несоответствий
    Args:
        metrics (dict): результаты работы с последней эпохи
    """
    fpr, tpr, thresholds = roc_curve(metrics['labels'], metrics['probas'])
    plt.plot([0, 1], [0, 1], linestyle='--', label='Random')
    plt.plot(fpr, tpr, linestyle='solid', label='Model')
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()
    cm = pd.DataFrame(confusion_matrix(metrics['labels'], metrics['preds']), 
                      index=['y=0', 'y=1'], columns=['y^=0', 'y^=1'])
    display(cm)

In [ ]:
def evaluate(g: dgl.DGLHeteroGraph, all_embeddings: torch.Tensor) -> None:
    """
    Оценивает аcсuracy на тестовом множестве
    Args:
        g (dgl.DGLHeteroGraph): граф для обучения
        all_embeddings (torch.Tensor): тензор эмбеддингов узлов
    """
    with g.local_scope():
        g.ndata['h'] = all_embeddings
        g.apply_edges(criterion.apply_edges, etype='gc')
        logits = g.edges['gc'].data['score']
        test_preds = (logits[g.edges['gc'].data['test_mask'], 0].sigmoid() > 0.5).long()
        test_true = torch.ones_like(test_preds)
        test_acc = (test_preds == test_true).sum() / len(test_true)
        print(f'Test_acc: {test_acc.item():.4f}')

In [ ]:
# загружаем конфигурационный файл
CONFIG = yaml.safe_load(open('CONFIG.yaml', encoding='utf8'))

# Загружаем данные об ОКВЭД
okved_parts = ['okved_class_', 'okved_subclass', 'okved_group', 'okved_subgroup', 'okved_type_']
okved_data = pd.read_csv(CONFIG['paths']['okved_data_save'],
                         index_col=0,
                         dtype={c: str for c in okved_parts})

# вспомогательные словари для маппинга ОКВЭДов в целые числа
idx_to_okved = okved_data['okved'].to_dict()
section_to_idx = {s: idx for idx, s in enumerate(okved_data['раздел'].unique())}
okved_to_section = okved_data[['okved', 'раздел']].set_index('okved')['раздел'].map(section_to_idx).to_dict()

# загружаем гетерограф
with open(CONFIG['paths']['okved_graph'], 'rb') as fp:
    g = pickle.load(fp)

# создаем и обучаем модель
device = 'cpu'
assert device == 'cpu'
dataloader = create_dataloader(g, n_layers=2)
model = create_model(in_feats=g.ndata['features'].shape[1],
                     n_hidden=128,
                     n_out=32,
                     n_layers=2,
                     dropout_p=0.25,
                     device=device)

criterion = HeteroDMCELoss(emb_size=model.n_classes, train_on_gc=True)

model, metrics = train(model, criterion, dataloader, g,
                       CONFIG['paths']['okved_embeddings_model'],
                       n_epochs=50, log_every=10, max_no_improvements=5)

# визуализируем результат
visualize_train_results(metrics)
all_embeddings = model.get_embeddings(g)['okved']
evaluate(g, all_embeddings)

# сохраняем эмбеддинги кодов ОКВЭД
names = np.array([idx_to_okved[idx] for idx in g.ndata['okved_idx'].numpy()])
name_emb = dict(zip(names, all_embeddings.tolist()))
with open(CONFIG['paths']['okved_embeddings'], 'wb') as fp:
    pickle.dump(name_emb, fp)

In [7]:
def draw_2d(embeddings_2d: np.array,
            g: dgl.DGLHeteroGraph,
            okved_data: pd.DataFrame,
            okved_to_section: dict,
            idx_to_okved: dict,
            xlim: tuple = None,
            ylim: tuple = None,
            figsize: tuple = (15, 5),
            annotate: bool = False,
            name_len: int = 20,
            hide_spins: tuple = None,
            node_size: int = 5) -> None:
    """
    Рисует проекции эмбеддингов на плоскости
    Args:
        embeddings_2d (np.array): массив эмбеддингов узлов
        g (dgl.DGLHeteroGraph): граф для обучения
        okved_data (pd.DataFrame): таблица с информацией об ОКВЭД
        okved_to_section (dict): маппинг код раздела ОКВЭД - номер кода
        idx_to_okved (dict): маппинг номер кода ОКВЭД - код
        xlim (tuple, optional): ограничения по оси х
        ylim (tuple, optional): ограничения по оси у
        figsize (tuple, optional): размер фигуры
        annotate (bool, optional): True, если нужно добавить подписи к точкам (лучше не применять на полном датасете)
        name_len (int, optional): максимальная длина названия кода
        hide_spins (tuple, optional): какие рамки скрывать
        node_size (int, optional): размер точки
    """

    def cut(s: str, ln: int = 10) -> str:
        if len(s) >= ln:
            return s[:ln - 3] + '...'
        return s

    colors = np.array([okved_to_section[idx_to_okved[idx]] for idx in g.ndata['okved_idx'].numpy()])
    names = np.array([idx_to_okved[idx] + ' ' + cut(okved_data.loc[idx, 'name'], name_len)
                      for idx in g.ndata['okved_idx'].numpy()])

    fig, ax = plt.subplots(figsize=figsize)
    if xlim:
        x_mask = (embeddings_2d[:, 0] >= xlim[0]) & (embeddings_2d[:, 0] <= xlim[1])
    else:
        x_mask = np.ones(len(embeddings_2d)).astype(bool)
    if ylim:
        y_mask = (embeddings_2d[:, 1] >= ylim[0]) & (embeddings_2d[:, 1] <= ylim[1])

    else:
        y_mask = np.ones(len(embeddings_2d)).astype(bool)

    mask = x_mask & y_mask
    embs = embeddings_2d[mask]

    colors = colors[mask]
    names = names[mask]
    ax.scatter(embs[:, 0], embs[:, 1], s=node_size, c=colors)
    ax.set_xlabel('$h_0(v)$')
    ax.set_ylabel('$h_1(v)$')
    if annotate:
        for (x, y), txt in zip(embs, names):
            ax.text(x, y, txt, rotation=45)
    if xlim:
        ax.set_xlim(*xlim)
    if ylim:
        ax.set_ylim(*ylim)
    if hide_spins is not None:
        for spin in hide_spins:
            ax.spines[spin].set_visible(False)


In [ ]:
embeddings_2d = TSNE(2, random_state=43).fit_transform(all_embeddings.detach())
g.ndata['x'] = torch.from_numpy(embeddings_2d[:, 0])
g.ndata['y'] = torch.from_numpy(embeddings_2d[:, 1])
draw_2d(embeddings_2d, g, okved_data, okved_to_section, idx_to_okved, annotate=False)